In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time 

In [2]:
url= 'https://www.qiyas.sa/en/Statistics-Data/Schools/Pages/DetResults.aspx'

#Create a new Safari browser object
driver = webdriver.Safari()
driver.implicitly_wait(100)

#Go to the website
driver.get(url)

##Extract data for Capabilities, Boys, Theoretical, Private##
    
#Click on 'Capabilities'
driver.find_element_by_xpath('/html/body/form/div[3]/div/div/section/div[2]/div/div[1]/article/div[2]/div[1]/div/div/div/div[1]/div/span/div[1]/div/table/tbody/tr[1]/td/div/div[1]/div').click()

#Click on 'Boys'
driver.find_element_by_xpath('/html/body/form/div[3]/div/div/section/div[2]/div/div[1]/article/div[2]/div[1]/div/div/div/div[1]/div/span/div[1]/div/table/tbody/tr[2]/td/div/div[1]').click() 

#Click on 'Theoretical'
driver.find_element_by_xpath('/html/body/form/div[3]/div/div/section/div[2]/div/div[1]/article/div[2]/div[1]/div/div/div/div[1]/div/span/div[1]/div/table/tbody/tr[3]/td/div/div[2]/div/label').click() 

#Click on 'Private'
driver.find_element_by_xpath('/html/body/form/div[3]/div/div/section/div[2]/div/div[1]/article/div[2]/div[1]/div/div/div/div[1]/div/span/div[1]/div/table/tbody/tr[4]/td/div/div[2]/div').click() 

#Click on 'View the order of all schools'
driver.find_element_by_xpath('/html/body/form/div[3]/div/div/section/div[2]/div/div[1]/article/div[2]/div[1]/div/div/div/div[1]/div/span/div[1]/div/table/tbody/tr[5]/td/span[1]/label').click() 

#Click on 'Submit'
driver.find_element_by_xpath('/html/body/form/div[3]/div/div/section/div[2]/div/div[1]/article/div[2]/div[1]/div/div/div/div[1]/div/span/div[2]/div').click()

In [3]:
#Retrieving table from first page 
time.sleep(5)
soup_cbsp= BeautifulSoup(driver.page_source, 'html.parser')

#Parse and extract data 
table= soup_cbsp.find_all(class_='table-responsive')[1]

rows=table.find_all('tr')

COLUMNS= ['General Rank', 'Ministerial Number', 'School Name', 
          'City', 'Region', 'School Type', 'Category', 'Average']

sub_df=[]
for row in rows:
    cells= row.select('td')
    cells=cells[0:8]
    sub_df.append([cell.text.strip() for cell in cells])

data_cbtp = pd.DataFrame(sub_df, columns=COLUMNS).drop([0,1], axis=0).reset_index(drop=True)
data_cbtp

,General Rank,Ministerial Number,School Name,City,Region,School Type,Category,Average
0,83,53849,ثانوية التطوير الأهلية,الدمام,AlSharquia,Domestic,تعليم عام بنين,67.88
1,110,11188,ثانوية التربية السعودية الأهلية,الرياض,Riyadh,Domestic,تعليم عام بنين,66.94
2,114,11413,ثانوية الامتياز الأهلية,الرياض,Riyadh,Domestic,تعليم عام بنين,66.79
3,115,40594,مدارس القصواء الأهلية - المرحلة الثانوية,المدينة المنورة,Madina,Domestic,تعليم عام بنين,66.77
4,149,147175,ثانوية اجيال الزرقاء,خب البريدي,Qassim,Domestic,تعليم عام بنين,65.42
5,167,40563,مدارس دار الهجرة الأهلية - المرحلة الثانوية,المدينة المنورة,Madina,Domestic,تعليم عام بنين,64.99
6,191,61538,ثانوية المناهل الأهلية بأبها,أبها,Asser,Domestic,تعليم عام بنين,64.52
7,260,40565,مدارس دار الهجرة الأهلية - المرحلة الثانوية (ل...,المدينة المنورة,Madina,Domestic,تعليم عام بنين,62.67
8,273,26607,ثانوية البشرى بالزاهر الأهلية,مكة المكرمة,Makkah,Domestic,تعليم عام بنين,62.43
9,293,76453,الشمال الأهلية الثانوية الليلية,تبوك,Tabuk,Domestic,تعليم عام بنين,61.93


In [4]:
while True:
    try:
        next_page_btn= driver.find_elements(By.CLASS_NAME, "Next")
        if not next_page_btn:
            print("No next button")
            break
        next_btn= next_page_btn[0]
        if next_btn.get_attribute("class")== "aspNetDisabled Next":
            print("No more pages left")
            break
        else:
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "Next"))).click()
        
            time.sleep(10)
       
            soup_pgs= BeautifulSoup(driver.page_source, 'html.parser')
        
            #Parse and extract data 
            table_pgs= soup_pgs.find_all(class_='table-responsive')[1]
        
            rows_pgs=table_pgs.find_all('tr')
        
            COLUMNS_pgs= ['General Rank', 'Ministerial Number', 'School Name', 
              'City', 'Region', 'School Type', 'Category', 'Average']
        
            sub_df_pgs=[]
            for row_pgs in rows_pgs:
                cells_pgs= row_pgs.select('td')
                cells_pgs=cells_pgs[0:8]
                sub_df_pgs.append([cell_pgs.text.strip() for cell_pgs in cells_pgs])
        
            data_pgs = pd.DataFrame(sub_df_pgs, columns=COLUMNS_pgs).drop([0,1], axis=0).reset_index(drop=True)
            data_cbtp=data_cbtp.append(data_pgs, ignore_index=True)
    except TimeoutException:
        time.sleep(10)
        print("Exception has been thrown. No more pages left")
        break
driver.quit()

No more pages left


In [5]:
data_cbtp

,General Rank,Ministerial Number,School Name,City,Region,School Type,Category,Average
0,83,53849,ثانوية التطوير الأهلية,الدمام,AlSharquia,Domestic,تعليم عام بنين,67.88
1,110,11188,ثانوية التربية السعودية الأهلية,الرياض,Riyadh,Domestic,تعليم عام بنين,66.94
2,114,11413,ثانوية الامتياز الأهلية,الرياض,Riyadh,Domestic,تعليم عام بنين,66.79
3,115,40594,مدارس القصواء الأهلية - المرحلة الثانوية,المدينة المنورة,Madina,Domestic,تعليم عام بنين,66.77
4,149,147175,ثانوية اجيال الزرقاء,خب البريدي,Qassim,Domestic,تعليم عام بنين,65.42
5,167,40563,مدارس دار الهجرة الأهلية - المرحلة الثانوية,المدينة المنورة,Madina,Domestic,تعليم عام بنين,64.99
6,191,61538,ثانوية المناهل الأهلية بأبها,أبها,Asser,Domestic,تعليم عام بنين,64.52
7,260,40565,مدارس دار الهجرة الأهلية - المرحلة الثانوية (ل...,المدينة المنورة,Madina,Domestic,تعليم عام بنين,62.67
8,273,26607,ثانوية البشرى بالزاهر الأهلية,مكة المكرمة,Makkah,Domestic,تعليم عام بنين,62.43
9,293,76453,الشمال الأهلية الثانوية الليلية,تبوك,Tabuk,Domestic,تعليم عام بنين,61.93


In [6]:
data_cbtp.to_csv('cap_boys_the_priv.csv', index=False)